In [1]:
!pip install deep-svdd

     |████████████████████████████████| 394.3MB 38kB/s 


In [1]:
%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)
from dsvdd import DeepSVDD
from dsvdd.utils import plot_most_normal_and_abnormal_images

TensorFlow 1.x selected.
1.15.2


In [2]:
from dsvdd import *
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np

In [3]:
def get_letter(cls='A'):
    data = pd.read_csv("/content/letter-recognition.data")
    train_data = data.sample(frac=0.7)
    test_data = data.drop(train_data.index)
    x_train = np.array(train_data.drop('Label',axis=1))
    y_train = np.array(train_data['Label'])
    x_test = np.array(test_data.drop('Label',axis=1))
    y_test = np.array(test_data['Label'])
    # x_train, y_train = d_train
    # x_test, y_test = d_test

    mask = y_train == cls

    x_train = x_train[mask]
    x_train = np.expand_dims(x_train / 255., axis=-1).astype(np.float32)
    x_test = np.expand_dims(x_test / 255., axis=-1).astype(np.float32)

    y_test = (y_test == cls).astype(np.float32)
    return x_train.reshape(len(x_train),4,4,1), x_test.reshape(len(x_test),4,4,1), y_test

In [4]:
import keras
def letter_lenet(H=32):
    model = keras.models.Sequential()

    model.add(keras.layers.Conv2D(8, (3, 3), padding='same', use_bias=False, input_shape=(4, 4, 1)))
    model.add(keras.layers.LeakyReLU(1e-2))
    model.add(keras.layers.BatchNormalization(epsilon=1e-4, trainable=False))
    model.add(keras.layers.MaxPool2D())

    model.add(keras.layers.Conv2D(4, (3, 3), padding='same', use_bias=False))
    model.add(keras.layers.LeakyReLU(1e-2))
    model.add(keras.layers.BatchNormalization(epsilon=1e-4, trainable=False))
    model.add(keras.layers.MaxPool2D())

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(H, use_bias=False))

    return model

Using TensorFlow backend.


In [19]:
data = pd.read_csv("/content/letter-recognition.data")

predictions = []

dict_int_to_cls ={}
dict_cls_to_int ={}

for i,cls in enumerate(np.array(data['Label'].unique())):

# cls = 'A'
    # print(i)
    dict_int_to_cls[i] = cls
    dict_cls_to_int[cls]=i
    tensorflow.reset_default_graph()
    keras_model = letter_lenet(32)
    svdd = DeepSVDD(keras_model, input_shape=(4, 4, 1), representation_dim=32,
                    objective='soft-boundary')
    # keras_model.summary()

    # get dataset
    X_train, X_test, y_test = get_letter(cls)

    # print(X_train.shape,X_test.shape,y_test.shape)
    # train DeepSVDD
    svdd.fit(X_train, X_test, y_test, epochs=10, verbose=True)

    # print(y_test.shape)
    # print(y_test[:10])
    # test DeepSVDD
    score = svdd.predict(X_test)
    # predictions.append(np.array(score))
    auc = roc_auc_score(y_test, -score)
    # print(cls+' AUROC: %.3f' % auc)
    predictions.append(np.array(auc))

    # plot_most_normal_and_abnormal_images(X_test, score)
    # plt.show()

# predictions = np.array(predictions)

100%|██████████| 5/5 [00:00<00:00, 224.71it/s]

Epoch:   0 AUROC: 0.679
Epoch:   1 AUROC: 0.674



100%|██████████| 5/5 [00:00<00:00, 193.11it/s]

Epoch:   2 AUROC: 0.664
Epoch:   3 AUROC: 0.660



100%|██████████| 5/5 [00:00<00:00, 216.23it/s]

...

Epoch:   4 AUROC: 0.464
Epoch:   5 AUROC: 0.464


100%|██████████| 5/5 [00:00<00:00, 189.09it/s]

Epoch:   6 AUROC: 0.464
Epoch:   7 AUROC: 0.464



100%|██████████| 5/5 [00:00<00:00, 219.90it/s]


Epoch:   8 AUROC: 0.464
Epoch:   9 AUROC: 0.464


In [21]:
print("Accuracy: ",np.array(predictions).mean())

0.4821970684029112
